# Building a Catalog

In [2]:
from classes.source_catalog import GetCnilCatalog

url = 'https://www.data.gouv.fr/api/1/organizations/534fff61a3a7292c64a77d59/catalog'
headers = {'accept': 'application/json'}
url_add = 'https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv'
instance1 = GetCnilCatalog(url, headers, url_add)
data = instance1.fetch_data_from_api()
data = data['@graph']
table_name = 'title'
download_url = 'downloadURL'
table_id = 'identifier' 
file_format= 'format'
last_update= 'modified' 
accessURL = '@id'
df_catalog = instance1.response_to_dataframe(data=data, table_name=table_name, download_url=download_url, table_id=table_id, file_format=file_format, last_update=last_update, accessURL=accessURL)
df_dataset = instance1.load_additional_info()
df_catalog = instance1.identify_datasets_info()
df_catalog = instance1.merge_additional_info()
instance1.save_to_csv('source_cnil_catalog')

Request is a success: 200
CSV file has been loaded to this path data/catalog/source_cnil_catalog_2024-02-10.csv


# Uploading Files to GCS

In [2]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv','data/catalog/opencnil-violationsdcpnotifiees-20230930.csv']
dest_folder = 'raw'
dest_blob = ['source_cnil_catalog_2024-02-10.csv', 'opencnil-violationsdcpnotifiees-20230930.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

Bucket already exists.
Raw file source_cnil_catalog_2024-02-10.csv uploaded to GCS successfully to 2024-02-10/raw/source_cnil_catalog_2024-02-10.csv.
Raw file opencnil-violationsdcpnotifiees-20230930.csv uploaded to GCS successfully to 2024-02-10/raw/opencnil-violationsdcpnotifiees-20230930.csv.


In [2]:
bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
url = ['https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv', 'https://www.data.gouv.fr/fr/datasets/r/0f678674-4327-4c4d-8819-b6f508b41d0e']
dest_folder = 'raw'
dest_blob = ['datasets.csv', 'plaintes.csv']
init2.download_and_upload_from_URLs(url, dest_folder, dest_blob)

NameError: name 'FromFileToGCS' is not defined

# Downloading from Catalog

In [1]:
from classes.download_catalog_content import DlCatalogContent

instance3 = DlCatalogContent('data/catalog/source_cnil_catalog_2024-02-10.csv')
instance3.get_tables()
instance3.zip_files()

Error when downloading table Organismes ayant désigné un(e) délégué(e) à la protection des données (DPD/DPO) : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Notifications à la CNIL de violations de données à caractère personnel : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Sanctions prononcées par la CNIL : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Traitements de données personnelles déclarés à la CNIL depuis le 25 mai 2018 : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Protection des données personnelles dans le monde : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Contrôles réalisés par la CNIL : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Mises en demeure prononcées p

# Prep data to upload to BQ

In [3]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv','data/raw_datasets/raw_datasets.zip']
dest_folder = 'raw'
init2.local_to_gcs(file_paths, dest_folder)

Bucket already exists.
Raw file source_cnil_catalog_2024-02-10.csv uploaded to GCS successfully to 2024-02-11/raw/source_cnil_catalog_2024-02-10.csv.
Raw file raw_datasets.zip uploaded to GCS successfully to 2024-02-11/raw/raw_datasets.zip.


In [1]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-11/raw/raw_datasets.zip'
output_folder_name = '2024-02-11/'+ 'prep' + '/'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
instance4.process_zip_file()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa6 in position 17: invalid start byte